PyTorch simulation of a Flush+Reload-style reconstruction on VGG16

What this does:
1) Build VGG16(+Softmax), log a ground-truth op sequence via forward hooks.
2) Simulate a probing process over framework ops with realistic durations & noise.
3) Denoise (per-function debounce) and parse with a small VGG FSM.
4) Report 8 attributes and block structure (convs per block).

This notebook simulates the DeepRecon Flush+Reload attack for architecture extraction on VGG16.

## Imports + utility functions (set_seed, count_attrs, etc.):

In [ ]:
import time
import random
from typing import List, Dict, Tuple
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import vgg16, VGG16_Weights

# ---------- utils ----------
def set_seed(s=7):
    random.seed(s); np.random.seed(s); torch.manual_seed(s)

def count_attrs(seq: List[str]) -> Dict[str,int]:
    return {
        "#convs":   sum(x=="Conv2D"   for x in seq),
        "#fcs":     sum(x=="FC"       for x in seq),
        "#softms":  sum(x=="Softmax"  for x in seq),
        "#relus":   sum(x=="ReLU"     for x in seq),
        "#mpools":  sum(x=="MaxPool"  for x in seq),
        "#apools":  sum(x=="AvgPool"  for x in seq),
        "#merges":  sum(x=="Merge"    for x in seq),
        "#biases":  sum(x=="BiasAdd"  for x in seq),
    }

def split_blocks_no_bias(seq: List[str]) -> List[List[str]]:
    """Split feature-extractor ops by MaxPool (ignore BiasAdd)."""
    blocks, cur = [], []
    for op in seq:
        if op == "BiasAdd":  # ignore for classic block view
            continue
        if op == "MaxPool":
            blocks.append(cur[:]); cur=[]
        else:
            cur.append(op)
    return blocks

# 1) Build VGG16 and record ground-truth ops

Build VGG16WithSoftmax, run one forward, record GT ops. Show the first 30 ops as output.

In [ ]:
class VGG16WithSoftmax(nn.Module):
    def __init__(self):
        super().__init__()
        base = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
        self.features = base.features
        self.avgpool = base.avgpool
        self.classifier = base.classifier
        self.softmax = nn.Softmax(dim=1)  # explicit Softmax to make it visible
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        x = self.softmax(x)
        return x

def op_name_from_module(m: nn.Module) -> str:
    if isinstance(m, nn.Conv2d):      return "Conv2D"
    if isinstance(m, nn.ReLU):        return "ReLU"
    if isinstance(m, nn.MaxPool2d):   return "MaxPool"
    if isinstance(m, nn.AvgPool2d):   return "AvgPool"
    if isinstance(m, nn.Linear):      return "FC"
    if isinstance(m, nn.Softmax):     return "Softmax"
    # treat everything else as None (ignored)
    return ""

def record_ops(model: nn.Module, x: torch.Tensor) -> List[str]:
    ops = []
    def make_hook(name):
        def hook(_m, _inp, _out):
            n = op_name_from_module(_m)
            if n: ops.append(n)
        return hook
    handles=[]
    for m in model.modules():
        if m is model: continue
        if op_name_from_module(m):
            handles.append(m.register_forward_hook(make_hook(repr(m))))
    model.eval()
    with torch.no_grad():
        _ = model(x)
    for h in handles: h.remove()
    # inject BiasAdd after each Conv/FC (simulating framework bias kernels)
    seq = []
    for op in ops:
        seq.append(op)
        if op in ("Conv2D","FC"):
            seq.append("BiasAdd")
    return seq

# 2) Simulate Flush+Reload probes with noise

Simulate noisy probe trace (simulate_trace), plot latency histogram (hits vs misses)

In [ ]:
def simulate_trace(gt_seq: List[str],
                   durations_us: Dict[str,int],
                   probe_interval_us: int = 40,
                   hit_prob_active: float = 0.98,
                   false_pos_rate: float = 0.002,
                   drop_rate: float = 0.01,
                   hit_cycles: Tuple[int,int]=(80,120),
                   miss_cycles: Tuple[int,int]=(280,360)) -> List[Tuple[int,str,int,int]]:
    """
    Returns list of (timestamp_ns, function, latency_cycles, is_hit)
    """
    # Build schedule (serialized ops)
    t = 0
    schedule = []
    for op in gt_seq:
        dur = durations_us.get(op, 120)
        schedule.append((op, t, t+dur))
        t += dur
    total_us = t

    monitor = ["Conv2D","FC","ReLU","MaxPool","AvgPool","Softmax","Merge","BiasAdd"]
    records = []
    now = 0
    idx = 0
    curr = schedule[idx] if idx < len(schedule) else None
    while now < total_us + 2000:
        # advance current
        while curr and now > curr[2]:
            idx += 1
            curr = schedule[idx] if idx < len(schedule) else None
        active = curr[0] if (curr and curr[1] <= now <= curr[2]) else None
        for f in monitor:
            if f == active:
                is_hit = (random.random() >= drop_rate) and (random.random() < hit_prob_active)
            else:
                is_hit = (random.random() < false_pos_rate)
            lat = random.randint(*(hit_cycles if is_hit else miss_cycles))
            records.append((int(now*1000), f, lat, int(is_hit)))
        now += probe_interval_us
    return records

# 3) Denoise + FSM parsing

Denoise the trace, show number of events before/after
FSM parsing (vgg_fsm_parse), show recovered ops

the FSM encodes the legal layer order of VGG (e.g., Conv → ReLU → Pool → …).
It consumes the noisy sequence of observed ops and only keeps a path that matches this grammar, which recovers the valid architecture

In [ ]:
def denoise(records: List[Tuple[int,str,int,int]],
            latency_hit_threshold: int = 180,
            per_func_debounce_ns: int = 1_200_000,
            plausible_ops = ("Conv2D","FC","ReLU","MaxPool","Softmax","BiasAdd")) -> List[Tuple[int,str]]:
    # recompute hit via latency
    rows = []
    for ts, f, lat, is_hit in records:
        hit = is_hit or (lat < latency_hit_threshold)
        if hit:
            rows.append((ts, f))
    # per-function debounce
    rows.sort(key=lambda x: (x[1], x[0]))
    kept = []
    last_t = {}
    for ts, f in rows:
        lt = last_t.get(f, -10**18)
        if ts - lt >= per_func_debounce_ns:
            kept.append((ts,f))
            last_t[f] = ts
    kept.sort(key=lambda x: x[0])
    # plausibility filter for VGG
    kept = [r for r in kept if r[1] in plausible_ops]
    return kept

def vgg_fsm_parse(events: List[Tuple[int,str]], convs_per_block: List[int]) -> List[str]:
    """
    Grammar:
      For each block b with k=convs_per_block[b]:
        repeat k times: Conv2D -> [BiasAdd]? -> ReLU
        then: MaxPool
      Classifier:
        (FC -> [BiasAdd]? -> ReLU) x2
        (FC -> [BiasAdd]? -> Softmax) x1
    """
    recovered = []
    state = "BLOCKS"
    b = 0
    c_in_block = 0
    need_relu = False
    bias_seen = False
    fc_count = 0
    expect_relu = False
    bias_seen_fc = False
    for ts, f in events:
        if state == "BLOCKS":
            if b < len(convs_per_block):
                k = convs_per_block[b]
                if f == "Conv2D" and not need_relu and c_in_block < k:
                    recovered.append("Conv2D")
                    need_relu = True
                    bias_seen = False
                elif f == "BiasAdd" and need_relu and not bias_seen:
                    recovered.append("BiasAdd")
                    bias_seen = True
                elif f == "ReLU" and need_relu:
                    recovered.append("ReLU")
                    need_relu = False
                    c_in_block += 1
                elif f == "MaxPool" and (c_in_block == k) and not need_relu:
                    recovered.append("MaxPool")
                    b += 1; c_in_block = 0
            if b == len(convs_per_block) and not need_relu:
                state = "CLS"
        elif state == "CLS":
            # three FC groups
            if f == "FC" and not expect_relu and fc_count < 3:
                recovered.append("FC")
                fc_count += 1
                expect_relu = True
                bias_seen_fc = False
            elif f == "BiasAdd" and expect_relu and not bias_seen_fc:
                recovered.append("BiasAdd")
                bias_seen_fc = True
            elif f in ("ReLU","Softmax") and expect_relu:
                want = "ReLU" if fc_count < 3 else "Softmax"
                if f == want:
                    recovered.append(f)
                    expect_relu = False
                # stop after Softmax accepted
                if fc_count == 3 and f == "Softmax":
                    break
    return recovered


: 

# 4) Main

Compare GT vs Recovered attributes in a table

In [ ]:
def main():
    set_seed(7)

    # Build model and get GT op sequence via hooks
    model = VGG16WithSoftmax()
    x = torch.randint(0, 256, (1, 3, 224, 224), dtype=torch.uint8).float()
    gt_ops = record_ops(model, x)

    # Expected VGG16 convs per block (derive from GT by pooling split for robustness)
    gt_blocks = split_blocks_no_bias(gt_ops)
    convs_per_block = [sum(1 for op in b if op=="Conv2D") for b in gt_blocks]
    # Fallback to canonical if something odd happens
    if convs_per_block != [2,2,3,3,3]:
        convs_per_block = [2,2,3,3,3]

    # Op durations (µs) — rough realism
    durations_us = {
        "Conv2D": 2200, "FC": 1000, "ReLU": 120, "MaxPool": 160,
        "AvgPool": 160, "Softmax": 120, "Merge": 140, "BiasAdd": 90
    }

    # Simulate probe trace with noise
    records = simulate_trace(
        gt_seq=gt_ops,
        durations_us=durations_us,
        probe_interval_us=40,
        hit_prob_active=0.98,
        false_pos_rate=0.002,
        drop_rate=0.01
    )

    # Denoise → chronological events
    events = denoise(records,
                     latency_hit_threshold=180,
                     per_func_debounce_ns=1_200_000,
                     plausible_ops=("Conv2D","FC","ReLU","MaxPool","Softmax","BiasAdd"))

    # FSM parse to recover valid execution
    recovered = vgg_fsm_parse(events, convs_per_block=convs_per_block)

    # Attributes + blocks (ignore Bias for the block view)
    gt_attrs = count_attrs(gt_ops)
    rec_attrs = count_attrs(recovered)
    rec_blocks = split_blocks_no_bias(recovered)
    rec_convs_per_block = [sum(1 for op in b if op=="Conv2D") for b in rec_blocks]

    # Pretty print
    print("\n=== Ground Truth (first 40 ops) ===")
    print(gt_ops[:40], " ...")
    print("GT attributes:", gt_attrs)
    print("GT convs per block:", convs_per_block)

    print("\n=== Recovered (first 40 ops) ===")
    print(recovered[:40], " ...")
    print("Recovered attributes:", rec_attrs)
    print("Recovered convs per block:", rec_convs_per_block)

    # Simple quality metrics
    def l1_attr_err(a,b):
        keys = sorted(a.keys())
        return sum(abs(a[k]-b.get(k,0)) for k in keys)
    attr_err = l1_attr_err(gt_attrs, rec_attrs)
    ok_blocks = (rec_convs_per_block == convs_per_block)
    print("\nL1 error over 8 attributes:", attr_err)
    print("Blocks match expected pattern [2,2,3,3,3]? ->", ok_blocks)

if __name__ == "__main__":
    main()